In [1]:
clean_up = True
%run stdPackages.ipynb
d['data'] = os.path.join(d['data'], 'IO2018')

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.


# Setup for production module, SmallGR

The notebook defines the nesting structure for the production module. Additional tweaks to the production data can be loaded here as well. At this stage, we only define the main nesting structure as a mapping ```nestProduction```.

### Settings: 

Load IO data from the general model and store a new one with data specific to the production module:

In [2]:
name = 'smallGR' # name of model
module = name+'_P' # name of module
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'IO_{name}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)
db = GpyDB(**{'name': 'db_'+module})

### Nesting

**Non-materials nest:**

In [3]:
mFull = pd.MultiIndex.from_tuples([('KELBM', 'RxE'),
                                   ('KELBM', 'KELB'),
                                   ('KELB','L'), ('KELB','KE'),
                                   ('KE','iK'), ('KE','E'),
                                   ('E','energy_input'), ('E','energy_F')], names = ['n','nn'])
E = pd.Index(['energy','energy_F'], name = 'n')
m = pyDatabases.cartesianProductIndex([db_IO.get('s_p'), mFull])

Replace upper-most level with name of sector:

In [4]:
df = m.to_frame(index=False) 
df.loc[df.n == 'KELBM','n'] = df.loc[df.n == 'KELBM', 's']
m = pd.MultiIndex.from_frame(df)

**Materials nest:**

For [s,n] in ```dImport[s,n,nn]```, we define a mapping from ```RxE```to an intermediate good ```RxEym_x``` that is again an aggregate of the domestic/foreign goods ```x,x_F```. We do this for any sectors except the energy goods:

In [5]:
df = adj.rc_pd(db_IO.get('dImport'), ('and', [db_IO.get('s_p'), ('not',E)]) # imports for all domestic production sectors, not E
              ).to_frame(index=False).assign(RxEym= lambda x: 'RxEym_'+x['n'], RxE = 'RxE', n = lambda x: x['n']+'_input') # add intermediate goods
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','RxEym']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','RxEym','n']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','RxEym','nn']]).rename(['s','n','nn']))

For [s,n] in ```dImport_dom[s,n]```, we define a mapping from ```RxE```to the domestic goods:

In [6]:
df = adj.rc_pd(db_IO.get('dImport_dom'), ('and', [db_IO.get('s_p'), ('not',E)])
              ).to_frame(index=False).assign(RxE = 'RxE', n = lambda x: x['n']+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','n']]).rename(['s','n','nn']))

For [s,n] in ```dImport_for[s,n]```, we define a mapping from ```RxE```to the forein goods:

In [7]:
df = adj.rc_pd(db_IO.get('dImport_for'), ('and', [db_IO.get('s_p'), ('not',E)])
              ).to_frame(index=False).assign(RxE = 'RxE')
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','n']]).rename(['s','n','nn']))

Add to database:

In [8]:
db['nestProduction'] = m 

### Export data:

In [9]:
db.export(repo = d['data'])